# Lab2 - MapReduce

- Họ và tên: Trịnh Hửu Thọ

- MSSV: 22110238

# A. Cài Hadoop

### 1. Tải Hadoop

Có thể chọn các phiên bản khác nhau ở https://downloads.apache.org/hadoop/common/

In [1]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz

--2025-11-10 15:07:44--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  21.8MB/s    in 34s     

2025-11-10 15:08:19 (20.6 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



In [2]:
ls /content/

hadoop-3.3.6.tar.gz  sample_data/


### 2. Unzip và copy

In [3]:
!tar -xzvf hadoop-3.3.6.tar.gz && cp -r hadoop-3.3.6/ /content/

Streaming output truncated to the last 5000 lines.
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/DeleteOpParam.Op.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/PutOpParam.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/RenameOptionSetParam.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/XAttrEncodingParam.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/GetOpParam.Op.ht

### 3. Thiết lập Hadoop's Java

- Thiết lập JAVA_HOME path và export nó vào hadoop-env.sh

In [4]:
JAVA_HOME = !readlink -f /usr/bin/java | sed "s:bin/java::"
java_home_text = JAVA_HOME[0]
java_home_text_command = f"$ {JAVA_HOME[0]} "
!echo export JAVA_HOME=$java_home_text >>/content/hadoop-3.3.6/etc/hadoop/hadoop-env.sh

In [5]:
# Install Java if not present (often needed in fresh Colab environments)
# -qq suppresses most output, but still shows if packages are already installed or errors
!apt-get update -qq && apt-get install -y openjdk-8-jdk-headless -qq

# Now, try to find JAVA_HOME. $(which java) should now return a path.
JAVA_HOME_PATH = !dirname $(dirname $(readlink -f $(which java)))
java_home_text = JAVA_HOME_PATH[0] if JAVA_HOME_PATH else "/usr/lib/jvm/java-8-openjdk-amd64" # Fallback if command still fails
java_home_text_command = f"$ {java_home_text} "
!echo export JAVA_HOME=$java_home_text >>/content/hadoop-3.3.6/etc/hadoop/hadoop-env.sh

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package java-common.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../0-java-common_0.72build2_all.deb ...
Unpacking java-common (0.72build2) ...
Selecting previously unselected package libpcsclite1:amd64.
Preparing to unpack .../1-libpcsclite1_1.9.5-3ubuntu1_amd64.deb ...
Unpacking libpcsclite1:amd64 (1.9.5-3ubuntu1) ...
Selecting previously unselected package libxtst6:amd64.
Preparing to unpack .../2-libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../3-openjdk-8-jre-headless_8u462-ga~us1-0ubuntu2~22.04.2_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u462-ga~us1-0ubuntu2~22.04.2) ...

### 4. Thiết lập Thiết lập các biến môi trường (environment variables)

In [6]:
import os
os.environ['HADOOP_HOME']="/content/hadoop-3.3.6"
os.environ['JAVA_HOME']=java_home_text

### 5. Chạy Hadoop

In [7]:
!$HADOOP_HOME/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in worker mode
hosts filename                   list of hosts to use in worker mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
  

# B. Hướng dẫn dùng HDFS và MapReduce

- Phần này liên quan đến các tác vụ với hệ thống file trong HDFS (như sao chép file từ thư mục cục bộ vào HDFS, truy xuất file từ HDFS, sao chép di chuyển file, tạo và xóa thư mục) và các tác vụ xử lý dữ liệu với MapReduce.

- Tài liệu tham khảo về `dfs`

  - https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html

- Một số lệnh `dfs` phổ biến:

<img src="https://imgur.com/OfzCZP2.png"/>

- Tham khảo về các lệnh command line phổ biến trên linux

  - https://www.linuxtrainingacademy.com/linux-commands-cheat-sheet/




### 1. Xem lệnh `hdfs dfs`

In [8]:
!$HADOOP_HOME/bin/hdfs dfs

Usage: hadoop fs [generic options]
	[-appendToFile [-n] <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum [-v] <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-concat <target path> <src path> <src path> ...]
	[-copyFromLocal [-f] [-p] [-l] [-d] [-t <thread count>] [-q <thread pool queue size>] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-crc] [-ignoreCrc] [-t <thread count>] [-q <thread pool queue size>] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] [-e] [-s] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] [-t <thread count>] [-q <thread pool queue size>] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-v] [-x] <path> ...]
	[-expunge [-immediate] [-fs <path>]]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-crc] [-ignoreCrc] [-t <thread c

### 2. Tạo thư mục (directory) trên local

-  Xem nội dung của directory trong HDFS

In [9]:
!$HADOOP_HOME/bin/hdfs dfs -ls

Found 4 items
drwxr-xr-x   - root root       4096 2025-11-05 14:33 .config
drwxr-xr-x   - 1000 1000       4096 2023-06-18 09:08 hadoop-3.3.6
-rw-r--r--   1 root root  730107476 2023-06-25 23:35 hadoop-3.3.6.tar.gz
drwxr-xr-x   - root root       4096 2025-11-05 14:33 sample_data


- Xem root directory trong HDFS

In [10]:
!$HADOOP_HOME/bin/hdfs dfs -ls /

Found 30 items
-rwxr-xr-x   1 root root          0 2025-11-10 15:06 /.dockerenv
-rw-r--r--   1 root root      17294 2024-07-10 22:45 /NGC-DL-CONTAINER-LICENSE
drwxr-xr-x   - root root       4096 2025-11-10 15:09 /bin
drwxr-xr-x   - root root       4096 2022-04-18 10:28 /boot
drwxr-xr-x   - root root       4096 2025-11-10 15:08 /content
-rw-r--r--   1 root root       4332 2024-07-10 22:46 /cuda-keyring_1.1-1_all.deb
drwxr-xr-x   - root root       4096 2025-11-05 14:43 /datalab
drwxr-xr-x   - root root        360 2025-11-10 15:06 /dev
drwxr-xr-x   - root root       4096 2025-11-10 15:09 /etc
drwxr-xr-x   - root root       4096 2022-04-18 10:28 /home
drwxr-xr-x   - root root       4096 2025-11-10 15:06 /kaggle
drwxr-xr-x   - root root       4096 2025-11-10 15:09 /lib
drwxr-xr-x   - root root       4096 2024-06-27 14:20 /lib32
drwxr-xr-x   - root root       4096 2024-06-27 14:24 /lib64
drwxr-xr-x   - root root       4096 2024-06-27 14:20 /libx32
drwxr-xr-x   - root root       4096 2024-06-

- Tạo thư mục tên `data` trên local

In [11]:
!mkdir data

- Kiểm tra lại xem thư mục đã tạo thành công chưa

In [12]:
ls

data/  hadoop-3.3.6/  hadoop-3.3.6.tar.gz  sample_data/


In [13]:
!$HADOOP_HOME/bin/hadoop fs -rm -r dir_name

rm: `dir_name': No such file or directory


### 4. Upload một file vào HDFS

- Tải file

In [14]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2025-11-10 15:09:36--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.grouplens.org/datasets/movielens/ml-100k.zip [following]
--2025-11-10 15:09:36--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  24.5MB/s    in 0.2s    

2025-11-10 15:09:36 (24.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



- Extract file

In [15]:
!unzip /content/ml-100k.zip -d /content/data

Archive:  /content/ml-100k.zip
   creating: /content/data/ml-100k/
  inflating: /content/data/ml-100k/allbut.pl  
  inflating: /content/data/ml-100k/mku.sh  
  inflating: /content/data/ml-100k/README  
  inflating: /content/data/ml-100k/u.data  
  inflating: /content/data/ml-100k/u.genre  
  inflating: /content/data/ml-100k/u.info  
  inflating: /content/data/ml-100k/u.item  
  inflating: /content/data/ml-100k/u.occupation  
  inflating: /content/data/ml-100k/u.user  
  inflating: /content/data/ml-100k/u1.base  
  inflating: /content/data/ml-100k/u1.test  
  inflating: /content/data/ml-100k/u2.base  
  inflating: /content/data/ml-100k/u2.test  
  inflating: /content/data/ml-100k/u3.base  
  inflating: /content/data/ml-100k/u3.test  
  inflating: /content/data/ml-100k/u4.base  
  inflating: /content/data/ml-100k/u4.test  
  inflating: /content/data/ml-100k/u5.base  
  inflating: /content/data/ml-100k/u5.test  
  inflating: /content/data/ml-100k/ua.base  
  inflating: /content/data/ml-10

- Tạo folder trong HDFS tên `ml-100k`

In [16]:
# YOUR CODE
!$HADOOP_HOME/bin/hadoop fs -mkdir ml-100k

- Copy file từ local tại `data/ml-100k` vào thư mục của HDFS tên `ml-100k` vừa mới tạo

In [17]:
# YOUR CODE
!$HADOOP_HOME/bin/hadoop fs -copyFromLocal /content/data/ml-100k/* ml-100k/

- Liệt kê các file trong mục của HDFS tên `ml-100k`

In [18]:
# YOUR CODE
!$HADOOP_HOME/bin/hadoop fs -ls ml-100k

Found 23 items
-rw-r--r--   1 root root       6750 2025-11-10 15:09 ml-100k/README
-rw-r--r--   1 root root        716 2025-11-10 15:09 ml-100k/allbut.pl
-rw-r--r--   1 root root        643 2025-11-10 15:09 ml-100k/mku.sh
-rw-r--r--   1 root root    1979173 2025-11-10 15:09 ml-100k/u.data
-rw-r--r--   1 root root        202 2025-11-10 15:09 ml-100k/u.genre
-rw-r--r--   1 root root         36 2025-11-10 15:09 ml-100k/u.info
-rw-r--r--   1 root root     236344 2025-11-10 15:09 ml-100k/u.item
-rw-r--r--   1 root root        193 2025-11-10 15:09 ml-100k/u.occupation
-rw-r--r--   1 root root      22628 2025-11-10 15:09 ml-100k/u.user
-rw-r--r--   1 root root    1586544 2025-11-10 15:09 ml-100k/u1.base
-rw-r--r--   1 root root     392629 2025-11-10 15:09 ml-100k/u1.test
-rw-r--r--   1 root root    1583948 2025-11-10 15:09 ml-100k/u2.base
-rw-r--r--   1 root root     395225 2025-11-10 15:09 ml-100k/u2.test
-rw-r--r--   1 root root    1582546 2025-11-10 15:09 ml-100k/u3.base
-rw-r--r--   1 roo

- Xem vài dòng đầu của file `ml-100k/README` với `-head`

In [19]:
# YOUR CODE
!$HADOOP_HOME/bin/hadoop fs -head ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any rese

- Xem vài dòng cuối của file `ml-100k/README` với `-tail`

In [20]:
# YOUR CODE
!$HADOOP_HOME/bin/hadoop fs -tail ml-100k/README

occupations.

u1.base    -- The data sets u1.base and u1.test through u5.base and u5.test
u1.test       are 80%/20% splits of the u data into training and test data.
u2.base       Each of u1, ..., u5 have disjoint test sets; this if for
u2.test       5 fold cross validation (where you repeat your experiment
u3.base       with each training and test set and average the results).
u3.test       These data sets can be generated from u.data by mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base    -- The data sets ua.base, ua.test, ub.base, and ub.test
ua.test       split the u data into a training set and a test set with
ub.base       exactly 10 ratings per user in the test set.  The sets
ub.test       ua.test and ub.test are disjoint.  These data sets can
              be generated from u.data by mku.sh.

allbut.pl  -- The script that generates training and test sets where
              all but n of a users ratings are in the training data.

mku.sh     -- A shell script to generate all the u

### 5. Hadoop streaming với Python mapper và reducer

- Trong phần này, chúng ta sử dụng Python để triển khai một ứng dụng Hadoop streaming cổ điển, đếm
tổng số lần xuất hiện của mỗi từ trong một tệp văn bản. Ứng dụng bao gồm một mapper và một reducer.

- Mapper đọc từng dòng từ một file trong HDFS khi được xử lý bằng lệnh `mapred streamming`,
tách các dòng thành các từ và ghi các cặp khóa-giá trị `word \t 1`.

- Tạo một file tên `mapper.py` với nội dung sau.

In [21]:
%%writefile mapper.py

#!/usr/bin/env python

import sys

# Read each line from STDIN
for line in sys.stdin:

   # Get the words in each line
   words = line.split()

   # Generate the count for each word
   for word in words:

      # Write the key-value pair to STDOUT to be processed by the reducer.
      # The key is anything before the first tab character and the value is
      # anything after the first tab character.
      print('{0}\t{1}'.format(word, 1))

Writing mapper.py


- Reducer sẽ đọc các cặp key-value được tạo bởi `mapper.py` và đếm số từ.

In [22]:
%%writefile reducer.py

#!/usr/bin/env python
import sys

curr_word = None
curr_count = 0

# Process each key-value pair from the mapper
for line in sys.stdin:

   # Get the key and value from the current line
   word, count = line.split('\t')

   # Convert the count to an int
   count = int(count)

   # If the current word is the same as the previous word, increment its
   # count, otherwise print the words count to STDOUT
   if word == curr_word:
      curr_count += count
   else:

      # Write word and its number of occurrences as a key-value pair to STDOUT
      if curr_word:
         print('{0}\t{1}'.format(curr_word, curr_count))

      curr_word = word
      curr_count = count

# Output the count for the last word
if curr_word == word:
   print('{0}\t{1}'.format(curr_word, curr_count))

Writing reducer.py


- Đổi quyền truy xuất vào file `mapper.py` và `reducer.py` với lệnh `chmod`.

In [23]:
!chmod 777 /content/mapper.py /content/reducer.py

- Test `mapper.py` và `reducer.py` trên local

In [24]:
!cat /content/ml-100k/README | python mapper.py | sort -k1,1 | python reducer.py | head -50

&	1
*	7
-	1
--	11
...,	1
=============================================	1
==============================================	5
|	31
0	1
1	1
1.	1
10	1
100,000	1
100000	1
1/1/1970	1
(1-5)	1
1682	2
19	3
1992,	1
1996.	1
1997	1
1998.	1
1999	1
1999.	1
19th,	1
20	3
2015),	1
2015.	1
22nd,	1
4,	1
5	1
5,	1
80%/20%	1
943	2
a	11
A	3
A.	1
A.,	1
about	2
ABOUT	1
accompanying	1
acknowledge	2
ACKNOWLEDGEMENTS	1
ACM	1
Action	1
Adventure	1
(age,	1
age	1
Al	1
Algorithmic	1


- Dùng Hadoop streaming để thực thi `mapper.py` và `reducer.py`.

**Tham khảo**: https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html

In [25]:
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
  -input /content/ml-100k/README \
  -output /content/output/ \
  -mapper "python /content/mapper.py" \
  -reducer "python /content/reducer.py"

2025-11-10 15:09:51,085 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-11-10 15:09:51,195 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-11-10 15:09:51,195 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-11-10 15:09:51,218 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-11-10 15:09:51,579 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-10 15:09:51,604 INFO mapreduce.JobSubmitter: number of splits:1
2025-11-10 15:09:51,852 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local273366563_0001
2025-11-10 15:09:51,853 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-11-10 15:09:52,127 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2025-11-10 15:09:52,132 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2025-11-10 15:09:52,133 INFO mapreduce.Job: Running job: job_local273366563_0001
2025-11-1

- Xem kết quả trong thư mục `/content/output/`

In [26]:
!$HADOOP_HOME/bin/hdfs dfs -ls output

Found 2 items
-rw-r--r--   1 root root          0 2025-11-10 15:09 output/_SUCCESS
-rw-r--r--   1 root root       4420 2025-11-10 15:09 output/part-00000


- In ra vài dòng đầu của `part-00000`

In [27]:
!$HADOOP_HOME/bin/hdfs dfs -cat /content/output/part-00000 | head

&	1
(1-5)	1
(December	1
(TiiS)	1
(age,	1
(movielens.umn.edu)	1
(movies);	1
(see	1
(where	1
*	7
cat: Unable to write to output stream.


### 6. Dùng `mrjob` để thực hiện MapReduce với Python

- Cài `mrjob`

Link: https://mrjob.readthedocs.io/en/latest/

In [28]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 8.4 MB/s eta 0:00:00


- Xem top 10 dòng của `u.data`

In [29]:
!head /content/ml-100k/u.data -n 10

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


Mỗi dòng gồm một bộ (`userID`, `movieID`, `rating`, `timestamp`) mô tả một `userid` đánh giá (`rating`) một `movieid` vào thời gian (`timestamp`) nào.

- Tạo một process để dùng MRJOB với Python

Ở chương trình bên dưới, ta sẽ thực hiện việc thống kê số lượng đánh giá (rating) của tất cả người dùng ứng với mỗi loại (từ 1-5).

In [30]:
%%writefile RatingBreakdown.py

#!/usr/bin/env python

# import modules
from mrjob.job import MRJob
from mrjob.step import MRStep

# create class inhereted from MRJob
class RatingBreakdown(MRJob):
  # assign steps, first mapper last reducer
  def steps(self):
    return [
            MRStep(mapper=self.mapper_get_rating,
                   reducer=self.reducer_count_ratings)
    ]

  # creating mapper, assigning attributes from dataset
  def mapper_get_rating(self, _, line):
    (userID, movieID, rating, timestamp) = line.split('\t')
    # assign like the key rating and assign each row value 1
    yield rating, 1

  # creating reducer, sum
  def reducer_count_ratings(self, key, values):
    # in function of each key we sum values
    yield key, sum(values)

if __name__ == '__main__':
  RatingBreakdown.run()

Writing RatingBreakdown.py


- Chạy process và xuất kết quả ra `results.txt`

In [31]:
!python RatingBreakdown.py /content/ml-100k/u.data > results.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/RatingBreakdown.root.20251110.151005.547723
Running step 1 of 1...
job output is in /tmp/RatingBreakdown.root.20251110.151005.547723/output
Streaming final output from /tmp/RatingBreakdown.root.20251110.151005.547723/output...
Removing temp directory /tmp/RatingBreakdown.root.20251110.151005.547723...


- Liệt kê nội dung `results.txt`

In [32]:
!cat results.txt

"5"	21201
"4"	34174
"1"	6110
"2"	11370
"3"	27145


# C. Thực hành

Dựa trên dữ liệu từ `u.data`, hãy thực thực các yêu cầu sau (với Hadoop streamming hoặc MrJob):

### 1. Cho biết mỗi người dùng thực hiện bao nhiêu đánh giá

- Lưu kết quả vào `q1.txt`

In [33]:
%%writefile UserRatingCount.py

#!/usr/bin/env python

# import modules
from mrjob.job import MRJob
from mrjob.step import MRStep

# create class inherited from MRJob
class UserRatingCount(MRJob):
  # assign steps, first mapper last reducer
  def steps(self):
    return [
            MRStep(mapper=self.mapper_get_user_id,
                   reducer=self.reducer_count_ratings)
    ]

  # creating mapper, assigning attributes from dataset
  def mapper_get_user_id(self, _, line):
    (userID, movieID, rating, timestamp) = line.split('\t')
    # assign like the key userID and assign each row value 1
    yield userID, 1

  # creating reducer, sum
  def reducer_count_ratings(self, user_id, values):
    # in function of each key (user_id) we sum values (number of ratings)
    yield user_id, sum(values)

if __name__ == '__main__':
  UserRatingCount.run()

Writing UserRatingCount.py


In [34]:
# Run the mrjob script and save the output to q1.txt
!python UserRatingCount.py /content/ml-100k/u.data > q1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/UserRatingCount.root.20251110.151008.120444
Running step 1 of 1...
job output is in /tmp/UserRatingCount.root.20251110.151008.120444/output
Streaming final output from /tmp/UserRatingCount.root.20251110.151008.120444/output...
Removing temp directory /tmp/UserRatingCount.root.20251110.151008.120444...


In [35]:
!cat q1.txt

"499"	102
"5"	175
"50"	24
"500"	225
"501"	73
"502"	37
"503"	160
"504"	251
"505"	112
"506"	242
"507"	58
"508"	87
"509"	33
"51"	23
"510"	30
"511"	24
"512"	21
"513"	22
"514"	194
"515"	40
"516"	21
"517"	37
"518"	73
"519"	49
"52"	56
"520"	23
"521"	150
"522"	30
"523"	100
"524"	306
"525"	54
"526"	68
"527"	137
"528"	53
"529"	43
"53"	28
"530"	45
"531"	30
"532"	274
"533"	260
"534"	80
"535"	218
"536"	163
"537"	490
"538"	81
"539"	56
"54"	65
"540"	63
"541"	133
"542"	135
"543"	197
"544"	31
"545"	162
"546"	59
"547"	23
"548"	155
"549"	25
"55"	21
"550"	40
"551"	334
"552"	84
"553"	100
"554"	114
"555"	52
"556"	44
"557"	53
"558"	20
"559"	76
"56"	187
"560"	101
"561"	357
"562"	72
"563"	30
"564"	34
"565"	35
"566"	149
"567"	155
"568"	81
"569"	70
"57"	106
"570"	22
"571"	20
"572"	20
"573"	51
"574"	46
"575"	26
"576"	36
"577"	188
"578"	24
"579"	74
"58"	154
"580"	47
"581"	28
"582"	62
"583"	27
"584"	24
"585"	80
"586"	166
"587"	98
"588"	225
"589"	43
"59"	382
"590"	48
"591"	84
"592"	360
"593"	156
"594"	25
"595"	96
"5

### 2. Cho biết đánh giá trung bình của mỗi người dùng

- Lưu kết quả vào `q2.txt`

In [48]:
%%writefile UserAverageRating.py

#!/usr/bin/env python

# import modules
from mrjob.job import MRJob
from mrjob.step import MRStep

# create class inherited from MRJob
class UserAverageRating(MRJob):  # Changed class name for clarity
  # assign steps, first mapper last reducer
  def steps(self):
    return [
            MRStep(mapper=self.mapper_get_user_id,
                   reducer=self.reducer_average_ratings)
    ]

  # creating mapper, assigning attributes from dataset
  def mapper_get_user_id(self, _, line):
    (userID, movieID, rating, timestamp) = line.split('\t')
    # Yield user_id and the rating as a float
    yield userID, float(rating)

  # creating reducer to calculate average
  def reducer_average_ratings(self, user_id, values):
    # Calculate the sum and count of ratings by iterating through the generator
    total_rating = 0
    count = 0
    for rating in values:
        total_rating += rating
        count += 1
    # Calculate the average and yield the result
    yield user_id, total_rating / count

if __name__ == '__main__':
  UserAverageRating.run()

Overwriting UserAverageRating.py


In [49]:
!python UserAverageRating.py /content/ml-100k/u.data > q2.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/UserAverageRating.root.20251110.151704.456535
Running step 1 of 1...
job output is in /tmp/UserAverageRating.root.20251110.151704.456535/output
Streaming final output from /tmp/UserAverageRating.root.20251110.151704.456535/output...
Removing temp directory /tmp/UserAverageRating.root.20251110.151704.456535...


In [50]:
!cat q2.txt

"5"	2.874285714285714
"50"	3.5416666666666665
"500"	3.3422222222222224
"501"	3.767123287671233
"502"	3.108108108108108
"503"	4.01875
"504"	3.6215139442231075
"505"	3.3660714285714284
"506"	3.6735537190082646
"507"	4.724137931034483
"508"	3.8275862068965516
"509"	2.515151515151515
"51"	3.5652173913043477
"510"	2.7666666666666666
"511"	3.75
"512"	4.333333333333333
"513"	4.363636363636363
"514"	3.8402061855670104
"515"	3.0
"516"	4.095238095238095
"517"	3.2972972972972974
"518"	3.643835616438356
"519"	4.122448979591836
"52"	4.267857142857143
"520"	3.4347826086956523
"521"	3.14
"522"	4.333333333333333
"523"	4.36
"524"	3.4934640522875817
"525"	3.2962962962962963
"526"	3.235294117647059
"527"	3.832116788321168
"528"	3.7169811320754715
"529"	3.604651162790698
"53"	3.8214285714285716
"530"	3.888888888888889
"531"	3.2333333333333334
"532"	4.12043795620438
"533"	3.3846153846153846
"534"	4.075
"535"	3.9357798165137616
"536"	3.871165644171779
"537"	2.8653061224489798
"538"	3.617283950617284
"539"	3

### 3. Cho biết người dùng thực hiện nhiều đánh giá nhất

- Lưu kết quả vào `q3.txt`

In [80]:
%%writefile UserWithMostRatings.py

from mrjob.job import MRJob
from mrjob.step import MRStep

class UserWithMostRatings(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_swap_key_value,
                   reducer=self.reducer_find_max)
        ]

    def mapper_swap_key_value(self, _, line):
        # The input lines are in the format "user_id"\trating_count
        user_id, rating_count = line.split('\t')
        # Yield a common key (None) and the rating_count and user_id as the value
        # We need to convert rating_count to a comparable type (int)
        yield None, (int(rating_count), user_id.strip('"'))


    def reducer_find_max(self, _, rating_user_pairs):
        # The reducer receives None as the key and an iterable of (rating_count, user_id) pairs.
        # We iterate through all pairs to find the maximum rating count.
        max_rating_count = -1
        user_with_max_ratings = None
        for rating_count, user_id in rating_user_pairs:
            if rating_count > max_rating_count:
                max_rating_count = rating_count
                user_with_max_ratings = user_id
        yield max_rating_count, user_with_max_ratings

if __name__ == '__main__':
    UserWithMostRatings.run()

Overwriting UserWithMostRatings.py


In [92]:
# Run the mrjob script on q1.txt and save the output to q3.txt
!python UserWithMostRatings.py q1.txt > q3.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/UserWithMostRatings.root.20251110.160007.877278
Running step 1 of 1...
job output is in /tmp/UserWithMostRatings.root.20251110.160007.877278/output
Streaming final output from /tmp/UserWithMostRatings.root.20251110.160007.877278/output...
Removing temp directory /tmp/UserWithMostRatings.root.20251110.160007.877278...


In [109]:
# Display the content of q3.txt
!cat q3.txt

737	"405"


### 4. Cho biết người dùng có đánh giá trung bình cao nhất

- Lưu kết quả vào `q4.txt`

In [124]:
# YOUR CODE
%%writefile UserHighestAverage.py
#!/usr/bin/env python3
from mrjob.job import MRJob
from mrjob.step import MRStep

class UserHighestAverage(MRJob):

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper_extract,
                reducer=self.reducer_avg
            ),
            MRStep(
                reducer=self.reducer_find_max
            )
        ]

    # Bước 1: Mapper — lấy userID và rating
    def mapper_extract(self, _, line):
        try:
            userID, movieID, rating, timestamp = line.split('\t')
            yield userID, float(rating)
        except:
            pass

    # Bước 1: Reducer — tính trung bình từng user → emit (avg, userID)
    def reducer_avg(self, user_id, ratings):
        total = 0
        count = 0
        for r in ratings:
            total += r
            count += 1
        avg = total / count
        yield None, (avg, user_id)   # emit để bước 2 tìm max dễ dàng

    # Bước 2: Reducer — tìm user có trung bình cao nhất
    def reducer_find_max(self, _, avg_user_pairs):
        yield max(avg_user_pairs)  # tuple (avg, userID), so max = avg lớn nhất


if __name__ == '__main__':
    UserHighestAverage.run()

Overwriting UserHighestAverage.py


In [127]:
!python UserHighestAverage.py /content/ml-100k/u.data > q4.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/UserHighestAverage.root.20251110.162247.029674
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/UserHighestAverage.root.20251110.162247.029674/output
Streaming final output from /tmp/UserHighestAverage.root.20251110.162247.029674/output...
Removing temp directory /tmp/UserHighestAverage.root.20251110.162247.029674...


In [128]:
!cat q4.txt

4.869565217391305	"849"


### 5. Cho biết mỗi bộ phim nhận được bao nhiêu đánh giá

- Lưu kết quả vào `q5.txt`

In [96]:
%%writefile MovieRatingCount.py

#!/usr/bin/env python

# import modules
from mrjob.job import MRJob
from mrjob.step import MRStep

# create class inherited from MRJob
class MovieRatingCount(MRJob):
  # assign steps, first mapper last reducer
  def steps(self):
    return [
            MRStep(mapper=self.mapper_get_movie_id,
                   reducer=self.reducer_count_ratings)
    ]

  # creating mapper, assigning attributes from dataset
  def mapper_get_movie_id(self, _, line):
    (userID, movieID, rating, timestamp) = line.split('\t')
    # assign movieID as the key and assign each row value 1
    yield movieID, 1

  # creating reducer, sum
  def reducer_count_ratings(self, movie_id, values):
    # in function of each key (movie_id) we sum values (number of ratings)
    yield movie_id, sum(values)

if __name__ == '__main__':
  MovieRatingCount.run()

Overwriting MovieRatingCount.py


In [98]:
!python MovieRatingCount.py /content/ml-100k/u.data > q5.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MovieRatingCount.root.20251110.160125.510378
Running step 1 of 1...
job output is in /tmp/MovieRatingCount.root.20251110.160125.510378/output
Streaming final output from /tmp/MovieRatingCount.root.20251110.160125.510378/output...
Removing temp directory /tmp/MovieRatingCount.root.20251110.160125.510378...


In [99]:
!cat q5.txt

"334"	64
"335"	21
"336"	43
"337"	18
"338"	91
"339"	47
"34"	7
"340"	189
"341"	11
"342"	52
"343"	124
"344"	55
"345"	65
"346"	126
"347"	137
"348"	27
"349"	31
"35"	11
"350"	41
"351"	20
"352"	26
"353"	14
"354"	72
"355"	41
"356"	97
"357"	264
"358"	143
"359"	18
"36"	13
"360"	10
"361"	10
"362"	28
"363"	47
"364"	37
"365"	48
"366"	47
"367"	170
"368"	31
"369"	55
"37"	8
"370"	39
"371"	67
"372"	34
"373"	39
"374"	11
"375"	23
"376"	24
"377"	13
"378"	101
"379"	43
"38"	120
"380"	116
"381"	100
"382"	111
"383"	31
"384"	69
"385"	208
"386"	87
"387"	65
"388"	28
"389"	27
"39"	87
"390"	10
"391"	59
"392"	68
"393"	192
"394"	12
"395"	56
"396"	54
"397"	12
"398"	26
"399"	89
"4"	209
"40"	57
"400"	18
"401"	76
"402"	170
"403"	201
"404"	101
"405"	344
"406"	49
"407"	43
"408"	112
"409"	70
"41"	37
"410"	162
"411"	163
"412"	93
"413"	55
"414"	62
"415"	25
"416"	64
"417"	73
"418"	129
"419"	178
"42"	148
"420"	81
"421"	106
"422"	26
"423"	300
"424"	19
"425"	85
"426"	32
"427"	219
"428"	121
"429"	97
"43"	40
"430"	93
"431"	153
"43

### 6. Cho biết mỗi bộ phim được đánh giá trung bình trên tất cả người dùng là bao nhiêu?

- Lưu kết quả vào `q6.txt`

In [100]:
%%writefile MovieAverageRating.py

#!/usr/bin/env python

# import modules
from mrjob.job import MRJob
from mrjob.step import MRStep

# create class inherited from MRJob
class MovieAverageRating(MRJob):
  # assign steps, first mapper last reducer
  def steps(self):
    return [
            MRStep(mapper=self.mapper_get_movie_rating,
                   reducer=self.reducer_average_rating)
    ]

  # creating mapper, assigning attributes from dataset
  def mapper_get_movie_rating(self, _, line):
    (userID, movieID, rating, timestamp) = line.split('\t')
    # Yield movieID and the rating as a float
    yield movieID, float(rating)

  # creating reducer to calculate average
  def reducer_average_rating(self, movie_id, values):
    # Calculate the sum and count of ratings by iterating through the generator
    total_rating = 0
    count = 0
    for rating in values:
        total_rating += rating
        count += 1
    # Calculate the average and yield the result
    yield movie_id, total_rating / count

if __name__ == '__main__':
  MovieAverageRating.run()

Writing MovieAverageRating.py


In [101]:
!python MovieAverageRating.py /content/ml-100k/u.data > q6.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MovieAverageRating.root.20251110.160237.221877
Running step 1 of 1...
job output is in /tmp/MovieAverageRating.root.20251110.160237.221877/output
Streaming final output from /tmp/MovieAverageRating.root.20251110.160237.221877/output...
Removing temp directory /tmp/MovieAverageRating.root.20251110.160237.221877...


In [102]:
!cat q6.txt

"335"	2.761904761904762
"336"	2.627906976744186
"337"	3.0
"338"	2.659340659340659
"339"	3.25531914893617
"34"	2.7142857142857144
"340"	3.5873015873015874
"341"	3.090909090909091
"342"	2.923076923076923
"343"	3.096774193548387
"344"	3.6545454545454548
"345"	3.4
"346"	3.642857142857143
"347"	3.510948905109489
"348"	3.2962962962962963
"349"	2.903225806451613
"35"	2.1818181818181817
"350"	3.1463414634146343
"351"	2.7
"352"	2.1153846153846154
"353"	2.4285714285714284
"354"	3.486111111111111
"355"	2.926829268292683
"356"	3.381443298969072
"357"	4.291666666666667
"358"	2.6153846153846154
"359"	3.5
"36"	2.1538461538461537
"360"	3.2
"361"	3.5
"362"	2.8214285714285716
"363"	2.723404255319149
"364"	2.675675675675676
"365"	3.2083333333333335
"366"	3.3404255319148937
"367"	3.458823529411765
"368"	1.903225806451613
"369"	2.6363636363636362
"37"	2.25
"370"	2.8205128205128207
"371"	3.1641791044776117
"372"	3.5294117647058822
"373"	2.8974358974358974
"374"	2.272727272727273
"375"	1.9565217391304348
"37

### 7. Cho biết bộ phim được người dùng đánh giá cao nhất (đánh giá trung bình trên tất cả người dùng)

- Lưu kết quả vào `q7.txt`

In [103]:
%%writefile MovieWithHighestAverageRating.py

from mrjob.job import MRJob
from mrjob.step import MRStep

class MovieWithHighestAverageRating(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_swap_key_value,
                   reducer=self.reducer_find_max)
        ]

    def mapper_swap_key_value(self, _, line):
        # The input lines are in the format "movie_id"\taverage_rating
        movie_id, average_rating = line.split('\t')
        # Yield a common key (None) and the average_rating and movie_id as the value
        # We need to convert average_rating to a comparable type (float)
        yield None, (float(average_rating), movie_id.strip('"'))

    def reducer_find_max(self, _, avg_rating_movie_pairs):
        # The reducer receives None as the key and an iterable of (average_rating, movie_id) pairs.
        # We iterate through all pairs to find the maximum average rating.
        max_average_rating = -1.0
        movie_with_highest_average_rating = None
        for average_rating, movie_id in avg_rating_movie_pairs:
            if average_rating > max_average_rating:
                max_average_rating = average_rating
                movie_with_highest_average_rating = movie_id
        yield max_average_rating, movie_with_highest_average_rating

if __name__ == '__main__':
    MovieWithHighestAverageRating.run()

Writing MovieWithHighestAverageRating.py


In [104]:
!python MovieWithHighestAverageRating.py q6.txt > q7.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MovieWithHighestAverageRating.root.20251110.160357.841101
Running step 1 of 1...
job output is in /tmp/MovieWithHighestAverageRating.root.20251110.160357.841101/output
Streaming final output from /tmp/MovieWithHighestAverageRating.root.20251110.160357.841101/output...
Removing temp directory /tmp/MovieWithHighestAverageRating.root.20251110.160357.841101...


In [105]:
!cat q7.txt

5.0	"1122"


### 8. Cho biết bộ phim nhận được nhiều đánh giá của người dùng nhất

- Lưu kết quả vào `q8.txt`

In [106]:
%%writefile MovieWithMostRatings.py

from mrjob.job import MRJob
from mrjob.step import MRStep

class MovieWithMostRatings(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_swap_key_value,
                   reducer=self.reducer_find_max)
        ]

    def mapper_swap_key_value(self, _, line):
        # The input lines are in the format "movie_id"\trating_count (from q5.txt)
        movie_id, rating_count = line.split('\t')
        # Yield a common key (None) and the rating_count and movie_id as the value
        # We need to convert rating_count to a comparable type (int)
        yield None, (int(rating_count), movie_id.strip('"'))

    def reducer_find_max(self, _, rating_movie_pairs):
        # The reducer receives None as the key and an iterable of (rating_count, movie_id) pairs.
        # We iterate through all pairs to find the maximum rating count.
        max_rating_count = -1
        movie_with_most_ratings = None
        for rating_count, movie_id in rating_movie_pairs:
            if rating_count > max_rating_count:
                max_rating_count = rating_count
                movie_with_most_ratings = movie_id
        yield max_rating_count, movie_with_most_ratings

if __name__ == '__main__':
    MovieWithMostRatings.run()

Writing MovieWithMostRatings.py


In [107]:
!python MovieWithMostRatings.py q5.txt > q8.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MovieWithMostRatings.root.20251110.160552.230907
Running step 1 of 1...
job output is in /tmp/MovieWithMostRatings.root.20251110.160552.230907/output
Streaming final output from /tmp/MovieWithMostRatings.root.20251110.160552.230907/output...
Removing temp directory /tmp/MovieWithMostRatings.root.20251110.160552.230907...


In [108]:
!cat q8.txt

583	"50"
